In [1]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Engine in Vertex AI - Chat history management

* This colab explains how to deal with chat history being stored in firestore.   
* Firestore reference : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/

## Init Vertex AI SDK

In [2]:
%pip install --quiet --user google-cloud-aiplatform[agent_engines,langchain,ag2]

In [3]:
# Note:  Need to restart the kernel
%pip install  --user --quiet google-cloud-firestore
%pip install  --user --quiet langchain-google-firestore
%pip install  --user --quiet langchain-google-vertexai

In [4]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-1.5-flash-002"
# MODEL_NAME = "gemini-2.0-flash-001"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

In [5]:
# @title Firestore configuration
DATABASE="chat-history"
COLLECTION="history"

In [6]:
# @title Create a bucket.
BUCKET_URI = f"gs://agent-0319"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [7]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [8]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://agent-0319/
No changes made to gs://agent-0319/


In [9]:
# @title Initialize Vertex AI with Staging Bucket.
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Firestore setting for chat history

Before executing the following cell, you have to set up firestore database in GCP Console.
* Create database : https://cloud.google.com/firestore/docs/manage-databases

### Helper method to get session history

In [10]:
# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    firestore_client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    chat_history = FirestoreChatMessageHistory(
                      client=firestore_client,
                      session_id=session_id,
                      collection=COLLECTION,
                      encode_message=False,
    )

    return chat_history

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")


def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [11]:
session_id = "conv-004"

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

[]


## Integrate reasoning engine and chat history

### Tools definition

In [12]:
# Define Google Search tool
from vertexai.generative_models import grounding, Tool

grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval(
        # Optional: For Dynamic Retrieval
        dynamic_retrieval_config=grounding.DynamicRetrievalConfig(
            mode=grounding.DynamicRetrievalConfig.Mode.MODE_DYNAMIC,
            dynamic_threshold=0.7,
        )
    )
)


### Agent - Local unit test

In [13]:
from vertexai.preview.reasoning_engines import LangchainAgent

# Agent Define
local_agent = LangchainAgent(
    model=MODEL_NAME,
    tools=[grounded_search_tool,],
    chat_history=get_session_history,  # function to get FirestoreChatMessageHistory
    agent_executor_kwargs={"return_intermediate_steps": False},
)

### Add chat come from interaction into chat history


In [19]:
from time import perf_counter

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(local_agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
print(chat_history.messages)

사용자: 안녕하세요.


1.0843009330001223

AI Agent : 안녕하세요!  무엇을 도와드릴까요?


------------------------------------ 
사용자: 현재 한국주변의 국제정세에 대해서 설명해주세요.


7.3829851830000734

AI Agent : 2025년 3월 19일 현재 한국 주변의 국제 정세는 매우 복잡하고 유동적입니다.  몇 가지 주요 요소들을 살펴보겠습니다.

**1. 미중 전략 경쟁 심화:** 미국과 중국 간의 전략적 경쟁은 지속적으로 심화되고 있으며, 이는 한반도를 포함한 동아시아 지역에 큰 영향을 미치고 있습니다. 양국은 경제, 기술, 군사 등 다방면에서 경쟁을 벌이고 있으며, 이러한 경쟁은 지역 안보와 안정에 불확실성을 더하고 있습니다.  특히, 대만 문제는 미중 갈등의 핵심 쟁점이며, 무력 충돌 가능성까지 제기되면서 지역 전체의 긴장을 고조시키고 있습니다.

**2. 러시아-우크라이나 전쟁 장기화:** 러시아-우크라이나 전쟁의 장기화는 국제 질서의 불확실성을 심화시키고, 세계 경제에 부정적인 영향을 미치고 있습니다.  전쟁은 에너지 가격 상승, 공급망 붕괴 등을 야기하며, 한국 경제에도 상당한 부담을 주고 있습니다.  또한, 러시아와 우크라이나 전쟁은 북한과의 관계에도 영향을 미칠 가능성이 있으며, 북한의 추가적인 도발 가능성을 높일 수 있다는 우려도 존재합니다.

**3. 북한의 핵 및 미사일 개발:** 북한은 핵무기와 미사일 기술을 계속 개발하고 있으며, 이는 한반도와 동북아시아 지역의 안보에 심각한 위협이 되고 있습니다. 북한의 핵 및 미사일 개발은 주변국들의 안보 우려를 증폭시키고 있으며,  국제사회의 대북 제재 강화로 이어지고 있습니다.  하지만 북한의 도발에도 불구하고, 북미 간 대화 가능성은 여전히 불투명한 상황입니다.

**4. 국제 질서의 파편화:**  미중 갈등과 러시아-우크라이나 전쟁 등으로 인해 국제 질서는 점점 더 파편화되고 있습니다.  국제 사회의 협력 체제가 약화되면서, 각국은 자국의 이익을 우선적으로 고려하는 경향을 보이고 있습니다. 이러한 국제 질서의 불확실성은 한국의 외교 정책에도 어려움을 야기하고 있습니다.

**5. 역내 국가 간 관계:** 한국은 미국, 중국, 일본, 러시아 등 주변 강대국들과 복잡한 관계를 맺고 있습니다.  이들 국가들 간의 관계 변화는 한국의 안보와 경제에 직접적인 영향을 미치기 때문에, 한국은 이들 국가들과의 관계를 신중하게 관리해야 합니다.


**결론적으로,** 현재 한국 주변의 국제 정세는 예측 불가능성이 높은 매우 불안정한 상황입니다.  미중 전략 경쟁, 러시아-우크라이나 전쟁 장기화, 북한의 핵 및 미사일 위협 등 다양한 요인들이 복합적으로 작용하고 있으며, 한국은 이러한 불안정한 국제 정세 속에서 안보와 경제적 안정을 유지하기 위해  능동적이고 유연한 외교 전략을 구사해야 합니다.  상기 내용은 2025년 3월 19일 현재 정보를 기반으로 작성되었으며,  상황 변화에 따라 내용이 달라질 수 있습니다.


------------------------------------ 
사용자: 종료
[HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요!  무엇을 도와드릴까요?\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 한국주변의 국제정세에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='2025년 3월 19일 현재 한국 주변의 국제 정세는 매우 복잡하고 유동적입니다.  몇 가지 주요 요소들을 살펴보겠습니다.\n\n**1. 미중 전략 경쟁 심화:** 미국과 중국 간의 전략적 경쟁은 지속적으로 심화되고 있으며, 이는 한반도를 포함한 동아시아 지역에 큰 영향을 미치고 있습니다. 양국은 경제, 기술, 군사 등 다방면에서 경쟁을 벌이고 있으며, 이러한 경쟁은 지역 안보와 안정에 불확실성을 더하고 있습니다.  특히, 대만 문제는 미중 갈등의 핵심 쟁점이며, 무력 충돌 가능성까지 제기되면서 지역 전체의 긴장을 고조시키고 있습니다.\n\n**2. 러시아-우크라이나 전쟁 장기화:** 러시아-우크라이나 전쟁의 장기화는 국제 질서의 불확실성을 심화시키고, 세계 경제에 부정적인 영향을 미치고 있습니다.  전쟁은 에너지 가격 상승, 공급망 붕괴 등을 야기하며, 한국 경제에도 상당한 부담을 주고 있습니다.  또한, 러시아와 우크라이나 전쟁은 북한과의 관계에도 영향을 미칠 가능성이 있으며, 북한의 추가적인 도발 가능성을 높일 수 있다는 우려도 존재합니다.\n\n**3. 북한의 핵 및 미사일 개발:** 북한은 핵무기와 미사일 기술을 계속 개발하고 있으며, 이는 한반도와 동북아시아 지역의 안보에 심각한 위협이 되고 있습니다. 북한의 핵 및 미사일 개발은 주변국들의 안보 우려를 증폭시키고 있으며,  국

In [20]:
session_id = "conv-004"
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요!  무엇을 도와드릴까요?\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국주변의 국제정세에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2025년 3월 19일 현재 한국 주변의 국제 정세는 매우 복잡하고 유동적입니다.  몇 가지 주요 요소들을 살펴보겠습니다.\n\n**1. 미중 전략 경쟁 심화:** 미국과 중국 간의 전략적 경쟁은 지속적으로 심화되고 있으며, 이는 한반도를 포함한 동아시아 지역에 큰 영향을 미치고 있습니다. 양국은 경제, 기술, 군사 등 다방면에서 경쟁을 벌이고 있으며, 이러한 경쟁은 지역 안보와 안정에 불확실성을 더하고 있습니다.  특히, 대만 문제는 미중 갈등의 핵심 쟁점이며, 무력 충돌 가능성까지 제기되면서 지역 전체의 긴장을 고조시키고 있습니다.\n\n**2. 러시아-우크라이나 전쟁 장기화:** 러시아-우크라이나 전쟁의 장기화는 국제 질서의 불확실성을 심화시키고, 세계 경제에 부정적인 영향을 미치고 있습니다.  전쟁은 에너지 가격 상승, 공급망 붕괴 등을 야기하며, 한국 경제에도 상당한 부담을 주고 있습니다.  또한, 러시아와 우크라이나 전쟁은 북한과의 관계에도 영향을 미칠 가능성이 있으며, 북한의 추가적인 도발 가능성을 높일 수 있다는 우려도 존재합니다.\n\n**3. 북한의 핵 및 미사일 개발:** 북한은 핵무기와 미사일 기술을 계속 개발하고 있으며, 이는 한반도와 동북아시아 지역의 안보에 심각한 위협이 되고 있습니다. 북한의 핵 및 미사일 개발은 주변국들의 안보 우려를 증폭시키고 있으며,  국제사회의 대북 제재 강화로 이어지고 있습니다.  하지만 북한의 도발에도 불구하

In [21]:
# Manual input for chat history if you want to add something usefule to chat history.
chat_history.add_user_message("what's your role?")
chat_history.add_ai_message("I'm a helpful assistant")

chat_history.messages

[HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요!  무엇을 도와드릴까요?\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국주변의 국제정세에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2025년 3월 19일 현재 한국 주변의 국제 정세는 매우 복잡하고 유동적입니다.  몇 가지 주요 요소들을 살펴보겠습니다.\n\n**1. 미중 전략 경쟁 심화:** 미국과 중국 간의 전략적 경쟁은 지속적으로 심화되고 있으며, 이는 한반도를 포함한 동아시아 지역에 큰 영향을 미치고 있습니다. 양국은 경제, 기술, 군사 등 다방면에서 경쟁을 벌이고 있으며, 이러한 경쟁은 지역 안보와 안정에 불확실성을 더하고 있습니다.  특히, 대만 문제는 미중 갈등의 핵심 쟁점이며, 무력 충돌 가능성까지 제기되면서 지역 전체의 긴장을 고조시키고 있습니다.\n\n**2. 러시아-우크라이나 전쟁 장기화:** 러시아-우크라이나 전쟁의 장기화는 국제 질서의 불확실성을 심화시키고, 세계 경제에 부정적인 영향을 미치고 있습니다.  전쟁은 에너지 가격 상승, 공급망 붕괴 등을 야기하며, 한국 경제에도 상당한 부담을 주고 있습니다.  또한, 러시아와 우크라이나 전쟁은 북한과의 관계에도 영향을 미칠 가능성이 있으며, 북한의 추가적인 도발 가능성을 높일 수 있다는 우려도 존재합니다.\n\n**3. 북한의 핵 및 미사일 개발:** 북한은 핵무기와 미사일 기술을 계속 개발하고 있으며, 이는 한반도와 동북아시아 지역의 안보에 심각한 위협이 되고 있습니다. 북한의 핵 및 미사일 개발은 주변국들의 안보 우려를 증폭시키고 있으며,  국제사회의 대북 제재 강화로 이어지고 있습니다.  하지만 북한의 도발에도 불구하